In [1]:
from ray.rllib.utils.framework import try_import_tf
tf1, tf, tfv = try_import_tf()

gpus = tf.config.list_physical_devices('GPU')
print(gpus)

[]


In [2]:
import numpy as np

from tqdm.auto import tqdm


from graphenv.examples.tsp.graph_utils import make_complete_planar_graph
from graphenv.graph_env import GraphEnv
from graphenv.examples.tsp.tsp_nfp_state import TSPNFPState
from graphenv.examples.tsp.tsp_nfp_model import TSPGNNModel

In [4]:
N = 40
G 
tsp_nfp_state = TSPNFPState(lambda: make_complete_planar_graph(N), max_num_neighbors=5)

## Check the greedy search heuristic baseline

In [8]:
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import greedy_tsp
tsp_approx = nx.approximation.traveling_salesman_problem

G = make_complete_planar_graph(N)
path = tsp_approx(G, cycle=True, method=greedy_tsp)
reward_baseline = -sum([G[path[i]][path[i + 1]]["weight"] for i in range(0, N)])
print(f"Networkx greedy reward: {reward_baseline:1.3f}")

Networkx greedy reward: -5.759


## Initialize a model without any trained weights

In [9]:
env = GraphEnv({
    "state": tsp_nfp_state,
    "max_num_children": G.number_of_nodes(),
})

model = TSPGNNModel._create_base_model(num_messages=1, embed_dim=32)

## Sample from the model's logit value predictions with a softmax

In [10]:
def sample_model():
    
    env.reset()
    obs = env.make_observation()
    done = False
    total_reward = 0

    while not done:
        values = model(obs['vertex_observations'])[0]
        masked_action_values = tf.where(
            obs['action_mask'][1:], values[1:, 0], values.dtype.min
        )
        action_probabilities = tf.nn.softmax(masked_action_values).numpy()
        action = np.random.choice(env.max_num_children, size=1, p=action_probabilities)[0]
        obs, reward, done, info = env.step(action)
        total_reward += reward
        
    return total_reward

In [11]:
[sample_model() for _ in tqdm(range(10))]

  0%|          | 0/10 [00:00<?, ?it/s]

[-8.09651494660714,
 -8.17159223100224,
 -7.107007948279428,
 -8.14660312876988,
 -8.35169345825159,
 -7.983293736413026,
 -7.449018970661648,
 -7.406426439637996,
 -7.5893263587106,
 -7.60277513497012]

## Create rllib agent

In [12]:
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog


ModelCatalog.register_custom_model('TSPGNNModel', TSPGNNModel)
register_env('GraphEnv', lambda config: GraphEnv(config))

config = {
    "env": 'GraphEnv',
    "env_config": {
        "state": tsp_nfp_state,
        "max_num_children": G.number_of_nodes(),
    },
    "model": {
        "custom_model": 'TSPGNNModel',
        "custom_model_config": {"num_messages": 1, "embed_dim": 32},
    },
    "num_workers": 1,
    "num_gpus": 0,
    "framework": "tf2",
    "eager_tracing": True,
}


from ray.rllib.agents import ppo



ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update(config)
agent = ppo.PPOTrainer(config=ppo_config)

(RolloutWorker pid=57858) 2022-05-17 20:24:08,072	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
(RolloutWorker pid=57858) 2022-05-17 20:24:08,072	WARNING env.py:120 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=57858) 2022-05-17 20:24:08,102	ERROR worker.py:449 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=57858, ip=127.0.0.1, repr=<ra

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=57858, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x1c499edc0>)
  File "/Users/pstjohn/mambaforge/envs/graphenv/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 507, in __init__
    check_env(self.env)
  File "/Users/pstjohn/mambaforge/envs/graphenv/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 65, in check_env
    check_gym_environments(env)
  File "/Users/pstjohn/mambaforge/envs/graphenv/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 156, in check_gym_environments
    raise ValueError(error)
ValueError: The observation collected from env.reset() was not  contained within your env's observation space. Its possible that There was a type mismatch, or that one of the sub-observations  was out of bounds: 

 reset_obs: {'action_mask': array([False,  True,  True, 

In [ ]:
env = GraphEnv({
        "state": tsp_nfp_state,
        "max_num_children": G.number_of_nodes(),
    })

def sample_ppo_action():
    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()

    while not done:
        action = agent.compute_single_action(obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
        
    return episode_reward

In [ ]:
[sample_ppo_action() for _ in tqdm(range(10))]